In [2]:
#!pip install sodapy

try:
    from sodapy import Socrata
except ModuleNotFoundError:
    !pip install sodapy

In [3]:
from sodapy import Socrata
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import datetime
import pandas as pd

In [4]:
# Spark session & context
spark = SparkSession.builder.master("local").getOrCreate()
sc = spark.sparkContext

In [5]:
client = Socrata(
    "data.sfgov.org", 
    "", # token
    username="",
    password="",
    timeout=10
)

In [6]:
has_true = True
offset = 0
limit = 100
count = 0

In [7]:
inicio = datetime.datetime.now()

# create an Empty DataFrame object
results_df = pd.DataFrame()

while has_true:
    
    results = client.get("wr8u-xric", where="incident_date >= '2023-01-01T00:00:00'", offset=offset, limit=limit) 
    # Convert to pandas DataFrame
    aux_df = pd.DataFrame.from_records(results)
    
    if len(results) < limit:
        has_true = False

    # concatenating results_df and aux_df along rows
    results_df = pd.concat([results_df, aux_df], axis=0)

    aux_df.drop(0,inplace=True)
    offset += limit
    count += 1

fim = datetime.datetime.now()
print(f"Execution time: {str(fim-inicio)}")
print(f"Interactions: {count}")

Tempo execução: 0:02:08.281069
Quantidade interações: 363


In [ ]:
# Verify if the requests keys have all cols

In [9]:
dataframe_columns = list(results_df.columns)

In [10]:
default_columns = [
    'incident_number', 'exposure_number', 'id', 'address', 'incident_date', 'call_number', 'alarm_dttm', 'arrival_dttm', 'close_dttm', 'city',
    'zipcode', 'battalion', 'station_area', 'suppression_units', 'suppression_personnel', 'ems_units', 'ems_personnel', 'other_units', 'other_personnel',
    'first_unit_on_scene', 'fire_fatalities', 'fire_injuries', 'civilian_fatalities', 'civilian_injuries', 'number_of_alarms', 'primary_situation', 
    'mutual_aid', 'action_taken_primary', 'action_taken_secondary', 'action_taken_other', 'detector_alerted_occupants', 'property_use', 'supervisor_district', 
    'neighborhood_district', 'point', 'estimated_contents_loss', 'area_of_fire_origin', 'ignition_cause', 'ignition_factor_primary', 'ignition_factor_secondary',
    'heat_source', 'item_first_ignited', 'human_factors_associated_with_ignition', 'estimated_property_loss', 'structure_type', 'structure_status', 'floor_of_fire_origin',
    'fire_spread', 'no_flame_spead', 'number_of_floors_with_minimum_damage', 'number_of_floors_with_significant_damage', 'number_of_floors_with_heavy_damage', 
    'number_of_floors_with_extreme_damage', 'detectors_present', 'detector_type', 'detector_operation', 'detector_effectiveness', 'detector_failure_reason',
    'automatic_extinguishing_system_present', 'automatic_extinguishing_sytem_type', 'automatic_extinguishing_sytem_perfomance', 'automatic_extinguishing_sytem_failure_reason',
    'number_of_sprinkler_heads_operating','box'
]

In [11]:
diff_columns = list(set(default_columns) - set(dataframe_columns))

In [12]:
sparkDF=spark.createDataFrame(results_df.astype(str))

In [13]:
for col in diff_columns:
    sparkDF = sparkDF.withColumn(col, F.lit(None))

In [ ]:
# Save on datalake...